# Importing Libraries

In [65]:
import os
import json
import pandas as pd
import numpy as np


In [66]:
folder_path = 'C:/Users/Prajwal/Desktop/BUAN_6341_Applied_Machine_Learning/Cricket/Tests_2011_Present/'
#folder_path = 'C:/Users/Prajwal/Desktop/BUAN_6341_Applied_Machine_Learning/Cricket/Sample/'
all_data = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    #print(file_path)
    
    with open(file_path, 'r', encoding='utf-8') as file: # Open and load each JSON file
        data = json.load(file)
        all_data.append(data)  # Append loaded data to the list

print(f"Loaded {len(all_data)} files successfully.")

Loaded 592 files successfully.


In [67]:
print(all_data[0]['info']['event']['name'])

South Africa in Australia Test Series


Viewing innings of some random match below.

In [68]:
all_data[2]['innings']

[{'team': 'South Africa',
  'overs': [{'over': 0,
    'deliveries': [{'batter': 'SC Cook',
      'bowler': 'MA Starc',
      'non_striker': 'D Elgar',
      'runs': {'batter': 0, 'extras': 0, 'total': 0}},
     {'batter': 'SC Cook',
      'bowler': 'MA Starc',
      'non_striker': 'D Elgar',
      'runs': {'batter': 0, 'extras': 0, 'total': 0}},
     {'batter': 'SC Cook',
      'bowler': 'MA Starc',
      'non_striker': 'D Elgar',
      'runs': {'batter': 4, 'extras': 0, 'total': 4}},
     {'batter': 'SC Cook',
      'bowler': 'MA Starc',
      'non_striker': 'D Elgar',
      'runs': {'batter': 0, 'extras': 0, 'total': 0}},
     {'batter': 'SC Cook',
      'bowler': 'MA Starc',
      'non_striker': 'D Elgar',
      'runs': {'batter': 0, 'extras': 0, 'total': 0}},
     {'batter': 'SC Cook',
      'bowler': 'MA Starc',
      'non_striker': 'D Elgar',
      'runs': {'batter': 0, 'extras': 0, 'total': 0}}]},
   {'over': 1,
    'deliveries': [{'batter': 'D Elgar',
      'bowler': 'JR Hazlew

### Checking if any file has missing event key.

In [69]:
for i in range(len(all_data)):
    event = all_data[i]['info'].get('event')
    
    if event:
        print(event)
    else:
        print(f"File {i} missing 'event' key.")

{'match_number': 1, 'name': 'South Africa in Australia Test Series'}
{'match_number': 2, 'name': 'South Africa in Australia Test Series'}
{'match_number': 3, 'name': 'South Africa in Australia Test Series'}
{'match_number': 1, 'name': 'Pakistan in Australia Test Series'}
{'match_number': 2, 'name': 'Pakistan in Australia Test Series'}
{'match_number': 3, 'name': 'Pakistan in Australia Test Series'}
{'match_number': 1, 'name': 'Bangladesh in New Zealand Test Series'}
{'match_number': 2, 'name': 'Bangladesh in New Zealand Test Series'}
{'match_number': 1, 'name': 'Pakistan in New Zealand Test Series'}
{'match_number': 2, 'name': 'Pakistan in New Zealand Test Series'}
{'match_number': 1, 'name': 'South Africa in New Zealand Test Series'}
{'match_number': 2, 'name': 'South Africa in New Zealand Test Series'}
{'match_number': 3, 'name': 'South Africa in New Zealand Test Series'}
{'match_number': 1, 'name': 'India in West Indies Test Series'}
{'match_number': 2, 'name': 'India in West Indies

In [70]:
total_matches = len(all_data)
total_matches

592

In [71]:
#umpires_call_true = df2[df2['IsUmpiresCall'] == True]

In [72]:
for i in range(total_matches):
    for j in range(len(all_data[i]['innings'])):
        inning_number = all_data[i]['innings'][j]
        team = inning_number['team']
        


The Umpires_Call decision can never be reversed i.e. 'Upheld' when IsUmpiresCall = True.

In [95]:
reviews_with_kind = []
review_data_missing = []

for i in range(total_matches):
    for j in range(len(all_data[i]['innings'])):
        inning_number = all_data[i]['innings'][j]
        team = inning_number['team']
        
        for over in inning_number['overs']:
            over_number = str(over['over'])
            for k, delivery in enumerate(over['deliveries']):
                if 'review' in delivery:
                    review_by = 'BattingTeam' if delivery['review']['by'] == team else 'BowlingTeam'
                    
                    #Handle wicket_type logic
                    wicket_type = 'Not Out'
                    wickets = delivery.get('wickets', [])

                    if wickets and isinstance(wickets, list):
                        wicket_type = wickets[0].get('kind', 'No decision')

                    reviews_with_kind.append({
                        'Match_ID': all_data[i]['info'].get('match_type_number'),
                        'Team1': all_data[i]['info']['teams'][0],
                        'Team2': all_data[i]['info']['teams'][1],
                        'Event': all_data[i]['info']['event']['name'],
                        'Year' : all_data[i]['info'].get('dates')[0][:4],
                        'City': all_data[i]['info'].get('city', 'N/A'),
                        'Venue': all_data[i]['info']['venue'],
                        'Innings': j + 1,
                        'Batting_Team': team,
                        'Over': f"{over_number}.{k + 1}",
                        'Bowler': delivery['bowler'],
                        'Batter': delivery['batter'],
                        'ReviewBy': review_by,
                        'IsUmpiresCall': delivery['review'].get('umpires_call', False),
                        'WicketType': wicket_type,
                        'Review_Decision': delivery['review']['decision'],
                        'Umpire_Name': delivery['review'].get('umpire', 'N/A')
                    })


In [96]:
reviews_with_kind_df = pd.DataFrame(reviews_with_kind)

In [97]:
reviews_with_kind_df

,Match_ID,Team1,Team2,Event,Year,City,Venue,Innings,Batting_Team,Over,Bowler,Batter,ReviewBy,IsUmpiresCall,WicketType,Review_Decision,Umpire_Name
0,2230,Australia,South Africa,South Africa in Australia Test Series,2016,Perth,Western Australia Cricket Association Ground,1,South Africa,12.2,PM Siddle,JP Duminy,BattingTeam,False,caught,struck down,NJ Llong
1,2230,Australia,South Africa,South Africa in Australia Test Series,2016,Perth,Western Australia Cricket Association Ground,1,South Africa,50.2,MA Starc,Q de Kock,BowlingTeam,False,Not Out,struck down,NJ Llong
2,2230,Australia,South Africa,South Africa in Australia Test Series,2016,Perth,Western Australia Cricket Association Ground,1,South Africa,55.4,JR Hazlewood,Q de Kock,BowlingTeam,False,Not Out,struck down,Aleem Dar
3,2230,Australia,South Africa,South Africa in Australia Test Series,2016,Perth,Western Australia Cricket Association Ground,2,Australia,41.3,KA Maharaj,SPD Smith,BattingTeam,False,lbw,struck down,Aleem Dar
4,2230,Australia,South Africa,South Africa in Australia Test Series,2016,Perth,Western Australia Cricket Association Ground,2,Australia,46.5,VD Philander,SE Marsh,BattingTeam,False,lbw,struck down,NJ Llong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603,2217,Sri Lanka,Australia,Warne-Muralitharan Trophy,2016,Colombo,Sinhalese Sports Club Ground,3,Sri Lanka,26.3,JM Holland,JK Silva,BowlingTeam,False,Not Out,struck down,S Ravi
5604,2217,Sri Lanka,Australia,Warne-Muralitharan Trophy,2016,Colombo,Sinhalese Sports Club Ground,3,Sri Lanka,79.3,NM Lyon,LD Chandimal,BowlingTeam,False,lbw,upheld,CB Gaffaney
5605,2217,Sri Lanka,Australia,Warne-Muralitharan Trophy,2016,Colombo,Sinhalese Sports Club Ground,4,Australia,17.3,MDK Perera,DA Warner,BowlingTeam,False,Not Out,struck down,S Ravi
5606,2217,Sri Lanka,Australia,Warne-Muralitharan Trophy,2016,Colombo,Sinhalese Sports Club Ground,4,Australia,17.4,MDK Perera,DA Warner,BowlingTeam,False,Not Out,struck down,CB Gaffaney


In [98]:
reviews_with_kind_df['Opposition'] = np.where(
    reviews_with_kind_df['Team1'] == reviews_with_kind_df['Batting_Team'],
    reviews_with_kind_df['Team2'],
    reviews_with_kind_df['Team1']
)

In [99]:
#Move 'Opposition' to the 9th position beside Batting_Team
cols = list(reviews_with_kind_df.columns)
cols.insert(9, cols.pop(cols.index('Opposition')))
reviews_with_kind_df = reviews_with_kind_df[cols]


In [100]:
reviews_with_kind_df.to_excel('reviews_with_kind.xlsx', sheet_name='Reviews', index=False)

In [94]:
for i in range(total_matches):
    print(all_data[i]['info'].get('match_type_number'))
    print(all_data[i]['info'].get('dates')[0][:4])

2230
2016
2233
2016
2236
2016
2240
2016
2242
2016
2245
2017
2246
2017
2248
2017
2234
2016
2237
2016
2253
2017
2255
2017
2257
2017
2207
2016
2211
2016
2215
2016
2218
2016
2210
2016
2214
2016
2225
2016
2227
2016
2221
2016
2222
2016
2223
2016
2262
2017
2264
2017
2266
2017
2268
2017
2270
2017
2271
2017
2274
2017
2232
2016
2235
2016
2238
2016
2239
2016
2241
2016
2249
2017
2224
2016
2226
2016
2229
2016
2228
2016
2231
2016
2250
2017
2251
2017
2256
2017
2258
2017
2282
2017
2285
2017
2288
2017
2289
2017
2291
2018
2276
2017
2277
2017
2297
2018
2298
2018
2300
2018
2302
2018
2259
2017
2260
2017
2261
2017
2252
2017
2254
2017
139
2017
2272
2017
2273
2017
2263
2017
2265
2017
2267
2017
2269
2017
2299
2018
2301
2018
2284
2017
2287
2017
2304
2018
2305
2018
2314
2018
2315
2018
2316
2018
2317
2018
2318
2018
2275
2017
2278
2017
2292
2018
2293
2018
2294
2018
2290
2017
2281
2017
2283
2017
2286
2017
2279
2017
2280
2017
2303
2018
2295
2018
2296
2018
2306
2018
2308
2018
2309
2018
2324
2018
2326
2018
2329
2018
2